In [15]:
import json
import os
import sys
import requests
import urllib.request
import numpy as np
import pandas as pd
import time

In [16]:
with open('data/secrets.json') as f:
    dict_api = json.load(f)
dict_weather = dict_api['weather']
api_key = dict_weather['API_KEY']
years = np.arange(2015, 2023)
months = np.arange(1, 13)

In [17]:
datas = []

In [18]:
for year in years:
    for month in months:
        url = f'http://apis.data.go.kr/1360000/SfcMtlyInfoService/getMmSumry?serviceKey={api_key}&numOfRows=9999&pageNo=9999&year={year}&month={month:0>2}&dataType=JSON'
        response = requests.get(url)
        rescode = response.status_code
        time.sleep(0.3)
        if (rescode == 200) and (response.json())['response']['header']['resultCode'] == '00':
            if response.json()['response']['body']['totalCount'] == 1:
                curr_datas = (response.json())['response']['body']['items']['item'][0]['month']['info']
                for data in curr_datas:
                    data['year'] = year
                    data['month'] = month
                datas.extend(curr_datas)
            else:
                raise ValueError('Several items exists.')

In [19]:
df_data = pd.DataFrame(datas)

In [20]:
df_data.head()

,maxcnt,daydur,avgtgmin,stnko,tamax,tmmax,avgcatot,tamin,mincnt,tmmin,...,pa,avghm,ta,stnid,avgte05,avgtamin,avgtamax,ps,year,month
0,0,61,-5.7,속초,12.2,05,4.0,-7.9,22,01,...,1019.8,52,2.1,90,null,-2.1,5.6,1022.1,2015,1
1,0,61,-14.3,철원,8.1,15,3.9,-16.3,31,08,...,1005.3,67,1.5,95,null,-10.0,2.1,1025.0,2015,1
2,0,61,-8.5,동두천,9.7,15,null,-13.0,30,03,...,1010.5,61,1.7,98,null,-7.4,4.0,1024.4,2015,1
3,0,65,-11.7,파주,8.8,15,null,-15.6,31,03,...,1021.2,61,null,99,null,-9.0,3.1,1025.0,2015,1
4,0,62,-10.7,대관령,7.0,05,4.5,-19.8,30,08,...,928.8,69,2.5,100,null,-11.2,0.2,1023.9,2015,1


In [97]:
list(df_data.columns)

['maxcnt',
 'daydur',
 'avgtgmin',
 'stnko',
 'tamax',
 'tmmax',
 'avgcatot',
 'tamin',
 'mincnt',
 'tmmin',
 'taavg',
 'sumssday',
 'pa',
 'avghm',
 'ta',
 'stnid',
 'avgte05',
 'avgtamin',
 'avgtamax',
 'ps',
 'year',
 'month']

In [21]:
df_data = df_data[[
    'year', 'month', 'avgtgmin', 'stnko', 'tamax', 'tmmax', 'avgcatot',
    'tamin', 'mincnt', 'tmmin', 'taavg', 'sumssday', 'pa', 'avghm', 'ta',
    'stnid', 'avgtamin', 'avgtamax', 'ps', 'maxcnt', 'daydur']]

In [22]:
df_data.to_csv('data/monthly_weather.csv')